In [1]:
# https://towardsdatascience.com/combining-numerical-and-text-features-in-deep-neural-networks-e91f0237eea4
"""combine meta data with text in keras"""

'combine meta data with text in keras'

In [32]:
import spacy
import gensim
nlp = spacy.load('en')
import pandas as pd
import numpy as np
import re
import demoji

In [33]:
file_path = "/home/ayush/Documents/personal/research/aaai/hostility_classification/data/constraint_Hindi_Train - Sheet1.csv"
data = pd.read_csv(file_path)
data.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile
3,4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"defamation,offensive"
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,non-hostile


In [34]:
# making labels
y = []
label_mapping = {'non-hostile':0, 'hate':1,'offensive':1,'fake':1,'defamation':1}
for labels in data['Labels Set']:
    temp = list(set([str(i)for i in [label_mapping[x] for x in labels.split(',')]]))
    temp = " ".join(temp)
    #print(temp)
    y.append(temp)
y = [int(x) for x in y]
y[:10]

[1, 0, 0, 1, 0, 1, 0, 0, 0, 0]

In [35]:
def hindi_tweet_cleaning(tweet):
    tweet = re.sub(r"(?:\@|https?\://)\S+", "", tweet)
    pat = re.compile(r"([',./@#(;_`:|\-)!])")
    tweet = pat.sub("", tweet)
    tweet_split_word_list = tweet.split()
    tweet_split_word_list_new = []
    #print(tweet_split_word_list)
    for word in tweet_split_word_list:
        if not word.isalpha():
            tweet_split_word_list_new.append(word)
    tweet = " ".join(tweet_split_word_list_new)
    tweet = demoji.replace(tweet,"")
    tweet = tweet.rstrip("\n")
    tweet = re.sub('\n', '', tweet)
    tweet = tweet.replace('\u200b', '')
    tweet = tweet.replace('\u200c', '')
    tweet = tweet.replace('\u200d', '')
    return tweet

In [36]:
all_tweets= []
for tweet in data['Post']:
    all_tweets.append(hindi_tweet_cleaning(tweet))

In [37]:
embedding_loc = "/home/ayush/Documents/personal/research/aaai/hostility_classification/fastText/cc.hi.300.bin"

In [8]:
import fasttext
import fasttext.util
ft = fasttext.load_model(embedding_loc)

In [38]:
X = []
for tweet in all_tweets:
    X.append(ft.get_sentence_vector(tweet))

In [39]:
X_train = X[:5500]
y_train = y[:5500]

X_test = X[5500:]
y_test = y[5500:]

In [40]:
from sklearn.svm import SVC

In [41]:
clf=SVC()

In [42]:
clf.fit(X_train,y_train)

SVC()

In [43]:
predictions=clf.predict(X_test)

In [44]:
from sklearn.metrics import accuracy_score

In [45]:
accuracy_score(y_test,predictions)

0.868421052631579

In [47]:
from sklearn import metrics
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.86      0.89      0.87       117
           1       0.88      0.85      0.86       111

    accuracy                           0.87       228
   macro avg       0.87      0.87      0.87       228
weighted avg       0.87      0.87      0.87       228



In [22]:
# testing data

In [23]:
file_path = "/home/ayush/Documents/personal/research/aaai/hostility_classification/data/Constraint_Hindi_Valid - Sheet1.csv"
data = pd.read_csv(file_path)

In [24]:
# making labels
y = []
label_mapping = {'non-hostile':0, 'hate':1,'offensive':1,'fake':1,'defamation':1}
for labels in data['Labels Set']:
    temp = list(set([str(i)for i in [label_mapping[x] for x in labels.split(',')]]))
    temp = " ".join(temp)
    #print(temp)
    y.append(temp)
y = [int(x) for x in y]
y[:10]

[0, 1, 0, 0, 0, 0, 0, 1, 0, 0]

In [25]:
all_tweets= []
for tweet in data['Post']:
    all_tweets.append(hindi_tweet_cleaning(tweet))

In [26]:
X = []
for tweet in all_tweets:
    X.append(ft.get_sentence_vector(tweet))

In [27]:
predictions=clf.predict(X)

In [28]:
accuracy_score(y,predictions)

0.8680641183723797

In [31]:
from sklearn import metrics
print(metrics.classification_report(y, predictions))

              precision    recall  f1-score   support

           0       0.86      0.90      0.88       435
           1       0.88      0.83      0.85       376

    accuracy                           0.87       811
   macro avg       0.87      0.87      0.87       811
weighted avg       0.87      0.87      0.87       811

